# Assignment 1:  Sentiment with Deep Neural Networks

Welcome to the first assignment of course 3. In this assignment, you will explore sentiment analysis using deep neural networks. 

In course 1, you implemented Logistic regression and Naive Bayes for sentiment analysis. However if you were to give your old models an example like:

<center> <span style='color:blue'> <b>This movie was almost good.</b> </span> </center>

Your model would have predicted a positive sentiment for that review. However, that sentence has a negative sentiment and indicates that the movie was not good. To solve those kinds of misclassifications, you will write a program that uses deep neural networks to identify sentiment in text. By completing this assignment, you will: 

- Understand how you can build/design a model using layers
- Train a model using a training loop
- Use a binary cross-entropy loss function
- Compute the accuracy of your model
- Predict using your own input

As you can tell, this model follows a similar structure to the one you previously implemented in the second course of this specialization. 
- Indeed most of the deep nets you will be implementing will have a similar structure. The only thing that changes is the model architecture, the inputs, and the outputs. Before starting the assignment, we will introduce you to the Google library `trax` that we use for building and training models.


Now we will show you how to compute the gradient of a certain function `f` by just using `  .grad(f)`. 

- Trax source code can be found on Github: [Trax](https://github.com/google/trax)
- The Trax code also uses the JAX library: [JAX](https://jax.readthedocs.io/en/latest/index.html)


# Part 1:  Import libraries and try out Trax

- Let's import libraries and look at an example of using the Trax library.

In [10]:
import os
import random as rnd

import numpy as np
import trax
from   trax import layers as t
from   trax.fastmath import grad
import trax.fastmath.numpy as fnp

from utils import get_all_tweets, Layer, load_tweets, process_tweet

#trax.supervised.trainer_lib.init_random_number_generators(31)

In [8]:
a = fnp.array(5.)
display(a)
type(a)

DeviceArray(5., dtype=float32)

jax.interpreters.xla.DeviceArray

In [3]:
def f(x):
    return x**2

In [4]:
print(f'f(a) for a={a}: {f(a)}')

f(a) for a=5.0: 25.0


The gradient (derivative) of function `f` with respect to its input `x` is the derivative of $x^2$.
- The derivative of $x^2$ is $2x$.  
- When x is 5, then $2x=10$.

You can calculate the gradient of a function by using `trax.fastmath.grad(fun=)` and passing in the name of the function.
- In this case the function you want to take the gradient of is `f`.
- The object returned (saved in `grad_f` in this example) is a function that can calculate the gradient of f for a given trax.fastmath.numpy array.

In [5]:
grad_f = grad(fun=f)
type(grad_f)

function

In [6]:
grad_calculation = grad_f(a) # f = x^2 -> d/dx = 2x
display(grad_calculation)

DeviceArray(10., dtype=float32)

# Part 2:  Importing the data

## 2.1  Loading in the data

Import the data set.  
- You may recognize this from earlier assignments in the specialization.
- Details of process_tweet function are available in utils.py file

In [17]:
DATA = '../../../../data/twitter_samples'

In [18]:
all_positive_tweets = get_all_tweets(DATA, 'positive')
all_negative_tweets = get_all_tweets(DATA, 'negative')

In [19]:
len(all_positive_tweets), len(all_negative_tweets)

(5000, 5000)

In [20]:
N_TRAIN = 4000
train_pos = all_positive_tweets[:N_TRAIN]
val_pos = all_positive_tweets[N_TRAIN:]
train_neg = all_negative_tweets[:N_TRAIN]
val_neg = all_negative_tweets[N_TRAIN:]

train_x = train_pos + train_neg
val_x = val_pos + val_neg
train_y = np.append(np.ones(len(train_pos)), np.zeros(len(train_neg)))
val_y = np.append(np.ones(len(val_pos)), np.zeros(len(val_neg)))

len(train_x), len(val_x)

(8000, 2000)

In [21]:
print('original:', train_pos[0])
print('processed:', process_tweet(train_pos[0]))

original: #FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)
processed: ['followfriday', 'top', 'engag', 'member', 'commun', 'week', ':)']


## 2.2  Building the vocabulary

Now build the vocabulary.
- Map each word in each tweet to an integer (an "index"). 
- The following code does this for you, but please read it and understand what it's doing.
- Note that you will build the vocabulary based on the training data. 
- To do so, you will assign an index to everyword by iterating over your training set.

The vocabulary will also include some special tokens
- `__PAD__`: padding
- `</e>`: end of line
- `__UNK__`: a token representing any word that is not in the vocabulary.